# Simple Agent with LangGraph

## What is a simple agent?

A **simple agent** is a program that uses an LLM (Large Language Model) to process messages and generate responses autonomously.

## Essential components:

1. **State**: Data structure that contains the message exchange history
2. **Node**: Function that processes the state and generates a response via the LLM
3. **Graph**: Orchestrator that defines the execution flow between nodes
4. **Entry point**: Initial node where processing begins
5. **End point (END)**: Indicates the end of execution

In this example, we create a basic conversational agent that:
- Receives a message from the user
- Processes it with the LLM
- Returns a response
- Terminates its execution

In [ ]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

In [ ]:
llm = ChatGroq(model="llama-3.1-8b-instant")

In [ ]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    messages: List[BaseMessage]

def chatbot_node(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    return {
        "messages": state["messages"] + [response]
    }


In [ ]:
graph = StateGraph(AgentState)

graph.add_node("chatbot", chatbot_node)
graph.set_entry_point("chatbot")
graph.add_edge("chatbot", END)

app = graph.compile()


In [ ]:
from IPython.display import display, Image

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
result = app.invoke({
    "messages": [
        HumanMessage(content="Explain LangGraph simply")
    ]
})

print(result["messages"][-1].content)